In [0]:
import requests
import json
import sys
import time
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType
from pyspark.sql.functions import udf, to_timestamp, unix_timestamp

In [0]:
# CONFIGURATION CENTRAL DE PROJETOS MICROSOFT
# client_id = '00000000-0000-0000-0000-000000000000'
# client_secret = '00000000000000000000000000.0000000000000'


In [0]:
session = requests.Session()

In [0]:
class SharepointAPI():
  def __init__(self, client_id='', client_secret='', page=''):
  #def __init__(self, page=''):
    self.tenant = 'tenant_name'
    self.tenant_id = '00000000-0000-0000-0000-000000000000'
    self.client_id = client_id + '@' + self.tenant_id
    self.client_secret = client_secret
    
    #print(client_id, client_secret)
    self.headers = {
      'Content-Type':'application/x-www-form-urlencoded'
    }
    if client_id != '' and client_secret != '': 
      self.get_credentials()
    self.page = page
    self.digest = ''
    

  def get_credentials(self):
    url = "https://accounts.accesscontrol.windows.net/00000000-0000-0000-0000-000000000000/tokens/OAuth/2"
    data = {
      'grant_type':'client_credentials',
      'resource': "00000000-0000-0000-0000-000000000000/" + self.tenant + ".sharepoint.com@" + self.tenant_id, 
      'client_id': self.client_id,
      'client_secret': self.client_secret,
    }
    r = requests.post(url, data=data, headers=self.headers)
    if not r.ok:
      raise Exception('AuthErrorException', r.text)
    json_data = json.loads(r.text)
    self.headers ={
      'Authorization': 'Bearer ' + json_data['access_token'],
      'Accept': 'application/json; odata=verbose',
      'Content-Type': 'application/json;odata=verbose'
    }
    self.header_project = self.headers
    
  def get_subpages(self, page):
    self.page = page
    url = f'https://{self.tenant}.sharepoint.com/sites/{self.page}/_api/web/webs/?$select=title,ServerRelativeUrl'
    r = requests.get(url, headers=self.headers)
#     print(r.text)
    return json.loads(r.text)
  
  def get_info_by_list_new(self, _list, _json, *args, ProjUID=False):
    '''
    Função busca dados de subpáginas de um tópico específico

    Parâmetros:
        `page`:
            Página da qual serão observadas as subpáginas
            Ex.: 'CentraldeProjetos'

        `_list`:
            Nome do tópico de interesse a ser buscado
            Ex.: 'Riscos'

        `_json`:
            Json contendo o conjunto de subpáginas que serão buscados

    Retorno:
        Dicionário de dados contendo informações coletadas dos tópicos em questão
    '''
    
    fields, desired_field = args if len(args) == 2 else [],[]
    res_dict = []
    i = 0
    id_pag = 1
    
    #print(_json)
    for item in _json['d']['results']:     
      
      subsite = item['ServerRelativeUrl'].replace(' ', '%20')
#       new_url = f"{base_url}{subsite}/_api/web/lists/getbytitle('{_list}')/items"
#       r = requests.get(new_url, headers=self.headers)
      
      subsite_info = self.get_info_new(_list, subsite=subsite, ProjUID=ProjUID, Title=item['Title'])
      res_dict.append(subsite_info)
    return [item for res in res_dict for item in res]
  
  def get_info_new(self, _list, *args, subsite='', ProjUID=False, Title=''):
  
    # Inicialização de variáveis
    fields, desired_field = args if len(list(args)) == 2 else ([],[])
    res_dict = []
    subsite = f'/sites/{self.page}' if subsite == '' else subsite
    #print(subsite) #debug
    url = f"https://{self.tenant}.sharepoint.com{subsite}/_api/web/lists/getbytitle('{_list}')/items"
    #print(url) #debug
    #time.sleep(1)
    r = requests.get(url, headers=self.headers)
    

    #print(r.text) #debug
    # Se a página possuir algum item na lista desejada
    if r.ok:
      #print(r)
      
      # Armazenando o resultado da requisição e verificando se há outras páginas na lista
      _j = json.loads(r.text)['d']
      next = '' if '__next' not in _j.keys() else _j['__next']
      pg_atual = _j['results']
      
      if ProjUID:
        project_url = f"https://{self.tenant}.sharepoint.com{subsite}/_api/Web/AllProperties"
        r = requests.get(project_url, headers = self.headers)
        proj_detail = json.loads(r.text)
        projeto_id = '' if 'MSPWAPROJUID' not in proj_detail['d'].keys() else proj_detail['d']['MSPWAPROJUID']

      # Enquanto houverem páginas seguintes
      while True:
        for item in pg_atual:

          list_fields = {}
          i = 0

          # Selecionando campos "brutos" (Antes do field as text)
          for i in range(len(fields)):
            list_fields[desired_field[i]] = item[fields[i]]
            i+=1
          
          #parte nova
#           print(item)
#           res_dict.append(item)
      
          new_url = item['FieldValuesAsText']['__deferred']['uri']
#           print(new_url)


          try:
            r = requests.get(new_url, headers=self.headers)

            # If the response was successful, no Exception will be raised
            r.raise_for_status()
          
          ### TRATAMENTO DE ERRO
          #except HTTPError as http_err:
          #  return "Erro durante a requisição da API do sharepoint: {}".format(http_err)
          
          except Exception as err:
            return "Erro durante a requisição da API do sharepoint: {}".format(err)
          ### TRATAMENTO DE ERRO
          
          # NÃO DEU ERRO, CONTINUA
          jsn_text = json.loads(r.text)['d']
            
          if ProjUID:
            jsn_text['ID_Projeto'] = projeto_id
            jsn_text['Projeto'] = Title

          # Adicionando os campos "brutos" selecionados
          for desired in desired_field:
            jsn_text[desired] = list_fields[desired]

          # adicionando item ao dicionário
          res_dict.append(jsn_text)

        # Verifica se há uma página seguinte
        if next == '':
          break
          
#         return res_dict

        # Preparando variáveis para a proxima iteração
        #print(next)
        url = next
        r = requests.get(url, headers=self.headers)
        _j = json.loads(r.text)['d']
        pg_atual = _j['results']
        next = '' if '__next' not in _j.keys() else _j['__next']

    return res_dict
  
  def get_info_new_byid(self, _listid, *args, subsite='', ProjUID=False, projeto_id=''):  
    # Inicialização de variáveis
    fields, desired_field = args if len(list(args)) == 2 else ([],[])
    res_dict = []
    subsite = f'/sites/{self.page}' if subsite == '' else subsite
    print(subsite) #debug
    url = f"https://{self.tenant}.sharepoint.com{subsite}/_api/web/lists(guid'{_listid}')/items"
    print(url) #debug
    r = requests.get(url, headers=self.headers)

    print(r.text) #debug
    print (r)
    # Se a página possuir algum item na lista desejada
    if r.ok:
      # Armazenando o resultado da requisição e verificando se há outras páginas na lista
      _j = json.loads(r.text)['d']
      next = '' if '__next' not in _j.keys() else _j['__next']
      pg_atual = _j['results']
      
      if ProjUID:
        project_url = f"https://{self.tenant}.sharepoint.com{subsite}/_api/Web/AllProperties"
        r = requests.get(project_url, headers = self.headers)
        proj_detail = json.loads(r.text)
        projeto_id = '' if 'MSPWAPROJUID' not in proj_detail['d'].keys() else proj_detail['d']['MSPWAPROJUID']

      # Enquanto houverem páginas seguintes
      while True:
        for item in pg_atual:

          list_fields = {}
          i = 0

          # Selecionando campos "brutos" (Antes do field as text)
          for i in range(len(fields)):
            list_fields[desired_field[i]] = item[fields[i]]
            i+=1
          
          #parte nova
#           print(item)
#           res_dict.append(item)
      
          new_url = item['FieldValuesAsText']['__deferred']['uri']
#           print(new_url)
  
  
          try:
            r = requests.get(new_url, headers=self.headers)

            # If the response was successful, no Exception will be raised
            r.raise_for_status()
               
          ### TRATAMENTO DE ERRO     
          except Exception as err:
            return "Erro durante a requisição da API do sharepoint: {}".format(err)
          ### TRATAMENTO DE ERRO
          
          # NÃO DEU ERRO, CONTINUA
          jsn_text = json.loads(r.text)['d']
          
          if ProjUID:
            jsn_text['ID_Projeto'] = projeto_id
            jsn_text['Projeto'] = Title

          # Adicionando os campos "brutos" selecionados
          for desired in desired_field:
            jsn_text[desired] = list_fields[desired]

          # adicionando item ao dicionário
          res_dict.append(jsn_text)

        # Verifica se há uma página seguinte
        if next == '':
          break
          
#         return res_dict

        # Preparando variáveis para a proxima iteração
        #print(next) #DEBUG
        url = next
        r = requests.get(url, headers=self.headers)
        _j = json.loads(r.text)['d']
        pg_atual = _j['results']
        next = '' if '__next' not in _j.keys() else _j['__next']

    return res_dict
  
  def create_df(self, data, rows, schema):
    lista = []
    for i in data:
      for r in rows:
        if r not in i.keys():
          i[r] = ''
      filtered_data = { key: i[key] for key in rows }
      lista.append(tuple(filtered_data.values()))
    return spark.createDataFrame(lista, schema)
  
 

In [0]:
def mes_num(mes):
  if mes == '':
    return 0
  meses = {'Jan':1,'Fev':2,'Mar':3,'Abr':4,'Mai':5,'Jun':6,'Jul':7,'Ago':8,'Set':9,'Out':10,'Nov':11,'Dez':12}
  return meses[mes]
  

udf_myFunction = udf(mes_num, IntegerType())

In [0]:
def mes_ano(mes, ano):
  data = None
  if not(mes == '' or ano == ''):
    if mes == 'Janeiro':
      mes = '01'
    elif mes == 'Fevereiro':
      mes = '02'
    elif mes == 'Março':
      mes = '03'
    elif mes == 'Abril':
      mes = '04'
    elif mes == 'Maio':
      mes = '05'
    elif mes == 'Junho':
      mes = '06'
    elif mes == 'Julho':
      mes = '07'
    elif mes == 'Agosto':
      mes = '08'
    elif mes == 'Setembro':
      mes = '09'
    elif mes == 'Outubro':
      mes = '10'
    elif mes == 'Novembro':
      mes = '11'
    else:
      mes = '12'
      
    data = "{}/{}".format(mes, ano)
  
  return data


func_mes_ano = udf(mes_ano, StringType())

In [0]:
def get_proj_ids(target_df, project_df):
  return target_df.join(project_df,col('Projeto') == col('NomedoProjeto'), how="left").select([col(xx) for xx in target_df.columns] + [col('IddeProjeto')])

In [0]:
def cast_double(df, columns):
  
  for column in columns:
    df = df.withColumn(column, round(df[column].cast(DoubleType()), 2))
  
  return df

In [0]:
def cast_float(df, columns):
  
  for column in columns:
    df = df.withColumn(column, round(df[column].cast(FloatType()), 2))
  
  return df

In [0]:
def rename_columns(df, nome_antigo, nome_novo):
  
  for i in range(len(nome_antigo)):
    df = df.withColumnRenamed(nome_antigo[i], nome_novo[i])
  
  return df

In [0]:
def remove_character(df, coluna, char, char2=''):
  
  # Caso queira retirar caracteres de uma lista de colunas
  if isinstance(coluna, list):
    for c in coluna:
      df = df.withColumn(c, regexp_replace(df[c], char, char2))
    return df

  # Caso seja apenas uma coluna
  return df.withColumn(coluna, regexp_replace(df[coluna], char, char2))

In [0]:
def cast_to_date(df, columns, date_format):
  if isinstance(columns, list):
    for c in columns:
      df = df.withColumn(c, to_timestamp(df[c], date_format))
    return df

  return df.withColumn(columns, to_timestamp(df[columns], date_format))
    

In [0]:
def cast_to_date_unix(df, cols, form):
  for c in cols:
    try:
      df = df.withColumn(c, unix_timestamp(c, form).cast('timestamp'))
    except NameError:
      print(f'Cannot cast column {c} to format {form}')
      return
  return df